In [1]:
from FunctionSet import *
# The NRPy_param_funcs module sets up global structures that manage free parameters within NRPy+
#import NRPy_param_funcs as par 
# The indexedexp module defines various functions for defining and managing indexed quantities like tensors and pseudotensors
import indexedexp as ixp
# The grid module defines various parameters related to a numerical grid or the dimensionality of indexed expressions
# For example, it declares the parameter DIM, which specifies the dimensionality of the indexed expression
#import grid as gri
#from outputC import *
import numpy as np

import sympy
from sympy import symbols, IndexedBase, Indexed, Idx #, preorder_traversal

A list of all evolved variables is initialized here.

In [2]:
list_vars = []

The conformal factor phi is initialized and appended to the evolved variables list.

In [3]:
symb_phi = symbols('phi')
phi = IndexedBase('state_fab'+str(symb_phi))[i,j,k,str(symb_phi)]

list_vars.append(str(symb_phi))

The conformally related spatial metric gbarLL (for gammabarLL) is initialized and added to the list.

In [4]:
symb_gbarLL = ixp.declarerank2('gbar','sym01')

gbarLL = ixp.zerorank2(DIM=3)
for itr1 in range(len(directions)):
    for itr2 in range(len(directions)):
        gbarLL[itr1][itr2] = IndexedBase('state_fab'+str(symb_gbarLL[itr1][itr2]))[i,j,k,str(symb_gbarLL[itr1][itr2])]

for itri in range(len(symb_gbarLL)):
    for itrj in range(itri,len(symb_gbarLL)):
        list_vars.append(str(symb_gbarLL[itri][itrj]))
        
symb_gbarLL = np.array(symb_gbarLL)

In [5]:
AMReXcode(sp.det(sp.Matrix(gbarLL)),list_vars)

'state_fab(i, j, k, Idx::gbar00)*state_fab(i, j, k, Idx::gbar11)*state_fab(i, j, k, Idx::gbar22) - state_fab(i, j, k, Idx::gbar00)*std::pow(state_fab(i, j, k, Idx::gbar12), 2) - std::pow(state_fab(i, j, k, Idx::gbar01), 2)*state_fab(i, j, k, Idx::gbar22) + 2*state_fab(i, j, k, Idx::gbar01)*state_fab(i, j, k, Idx::gbar02)*state_fab(i, j, k, Idx::gbar12) - std::pow(state_fab(i, j, k, Idx::gbar02), 2)*state_fab(i, j, k, Idx::gbar11);'

We need the extrinsic curvature scalar treated as an independent variable to be evolved and added to the list.

In [6]:
symb_Ksclr=symbols('Ksclr')
Ksclr = IndexedBase('state_fab'+str(symb_Ksclr))[i,j,k,str(symb_Ksclr)]

list_vars.append(str(symb_Ksclr))

The conformally related traceless part of the extrinsic curvature is initialized and added to the list.

In [7]:
symb_AbarLL = ixp.declarerank2('Abar','sym01')

AbarLL = ixp.zerorank2(DIM=3)
for itr1 in range(len(directions)):
    for itr2 in range(len(directions)):
        AbarLL[itr1][itr2] = IndexedBase('state_fab'+str(symb_AbarLL[itr1][itr2]))[i,j,k,str(symb_AbarLL[itr1][itr2])]

for itri in range(len(symb_AbarLL)):
    for itrj in range(itri,len(symb_AbarLL)):
        list_vars.append(str(symb_AbarLL[itri][itrj]))
        
symb_AbarLL = np.array(symb_AbarLL)

Trace of the conformal connection coefficients, treated as independent and evolved variables, and thus added to the variable list.

In [8]:
symb_GambarU = ixp.declarerank1('GambarU')
GambarU = ixp.zerorank1(DIM=3)

for itr in range(len(directions)):
    GambarU[itr] = IndexedBase('state_fab'+str(symb_GambarU[itr]))[i,j,k,str(symb_GambarU[itr])]

for itr in range(len(symb_GambarU)):
    list_vars.append(str(symb_GambarU[itr]))

symb_GambarU = np.array(symb_GambarU)

The gauge variable lapse function alpha is treated as an evolved variable in the 1+log gauge. Also added to the list.

In [9]:
symb_alpha = symbols('alpha')
alpha = IndexedBase('state_fab'+str(symb_alpha))[i,j,k,str(symb_alpha)]

list_vars.append(str(symb_alpha))

The shift beta and B will be treated in the gamma driver scheme, and are thus evolved.  Both added to the list.

In [10]:
symb_betaU = ixp.declarerank1('betaU')
betaU = ixp.zerorank1(DIM=3)

for itr in range(len(directions)):
    betaU[itr] = IndexedBase('state_fab'+str(symb_betaU[itr]))[i,j,k,str(symb_betaU[itr])]

for itr in range(len(symb_betaU)):
    list_vars.append(str(symb_betaU[itr]))

symb_betaU = np.array(symb_betaU)

In [11]:
symb_BU = ixp.declarerank1('BU')
BU = ixp.zerorank1(DIM=3)

for itr in range(len(directions)):
    BU[itr] = IndexedBase('state_fab'+str(symb_BU[itr]))[i,j,k,str(symb_BU[itr])]

for itr in range(len(symb_BU)):
    list_vars.append(str(symb_BU[itr]))

symb_BU = np.array(symb_BU)

The inverse metric gbarUU will be defined and calculated before inserting them into the evolved equations.  This reduces the number of calls to variables in memory.

In [12]:
symb_gbarUU = ixp.declarerank2('gbarinv','sym01')
#for itri in range(len(name_gUU)):
 #   for itrj in range(itri,len(name_gUU)):
  #      var_names.append(str(name_gUU[itri][itrj]))
symb_gbarUU = np.array(symb_gbarUU)

Defining the connection coefficents for both the barred and unbarred spatial metrics.

In [13]:
DcgbarLLL = ixp.zerorank3(3)
GammabarULL = ixp.zerorank3(3)
GammabarLLL=ixp.zerorank3(3)

for itri in range(len(directions)):
    for itrj in range(len(directions)):
        for itrk in range(len(directions)):
            DcgbarLLL[itri][itrj][itrk] = der(symb_gbarLL[itri][itrj],itrk)
            
for itri in range(len(directions)):
    for itrj in range(len(directions)):
        for itrk in range(len(directions)):
            for itrl in range(len(directions)):
                GammabarULL[itrk][itri][itrj] += 1/2*symb_gbarUU[itrk][itrl]*(DcgbarLLL[itrl][itri][itrj]+DcgbarLLL[itrl][itrj][itri]-DcgbarLLL[itri][itrj][itrl])
                
for itri in range(len(directions)):
    for itrj in range(len(directions)):
        for itrk in range(len(directions)):
            for itrl in range(len(directions)):
                GammabarLLL[itri][itrj][itrk]+= symb_gbarLL[itri][itrl]*GammabarULL[itrl][itrj][itrk]
                
GammaULL = ixp.zerorank3(3)
for i in range(3):
    for j in range(3):
        for k in range(3):
            GammaULL[i][j][k] += GammabarULL[i][j][k] + 2*(sp.eye(3)[i,j]*der(symb_phi,k)+sp.eye(3)[i,k]*der(symb_phi,j))
            for l in range(3):
                GammaULL[i][j][k] += -2*symb_gbarLL[j][k]*symb_gbarUU[i][l]*der(symb_phi,l)

Defining the Ricci curvature tensor.  It is split (as in standard BSSN convention) into the barred part and the phi part.

In [14]:
RbarLL = ixp.zerorank2(3)

for i in range(len(directions)):
    for j in range(i,len(directions)):
        for l in range(len(directions)):
            for m in range(l,len(directions)):
                RbarLL[i][j] += -symb_gbarUU[l][m]*der(der(symb_gbarLL[i][j],l),m)
                RbarLL[j][i] += -symb_gbarUU[l][m]*der(der(symb_gbarLL[i][j],l),m)
                for k in range(len(directions)):
                    RbarLL[i][j] += 2*symb_gbarUU[l][m]*(GammabarULL[k][l][i]*GammabarLLL[j][k][m]+GammabarULL[k][l][j]*GammabarLLL[i][k][m]+GammabarULL[k][i][m]*GammabarLLL[k][l][j])
                    RbarLL[j][i] += 2*symb_gbarUU[l][m]*(GammabarULL[k][l][i]*GammabarLLL[j][k][m]+GammabarULL[k][l][j]*GammabarLLL[i][k][m]+GammabarULL[k][i][m]*GammabarLLL[k][l][j])
        for  k in range(len(directions)):
            RbarLL[i][j]+= 1/2*(symb_gbarLL[k][i]*der(symb_GambarU[k],j)+symb_gbarLL[k][j]*der(symb_GambarU[k],i))+symb_GambarU[k]*1/2*(GammabarLLL[i][j][k]+GammabarLLL[j][i][k])
            RbarLL[j][i]+= 1/2*(symb_gbarLL[k][i]*der(symb_GambarU[k],j)+symb_gbarLL[k][j]*der(symb_GambarU[k],i))+symb_GambarU[k]*1/2*(GammabarLLL[i][j][k]+GammabarLLL[j][i][k])

In [15]:
RphiLL = ixp.zerorank2(3)
DbarDbarphi = ixp.zerorank2(3)

for i in range(len(directions)):
    for j in range(i,len(directions)):
        DbarDbarphi[i][j] += der(der(symb_phi,i),j)
        DbarDbarphi[j][i] += der(der(symb_phi,i),j)
        for k in range(len(directions)):
            DbarDbarphi[i][j] += GammabarULL[k][i][j]*der(symb_phi,k)
            DbarDbarphi[j][i] += GammabarULL[k][i][j]*der(symb_phi,k)

for i in range(len(directions)):
    for j in range(i,len(directions)):
        RphiLL[i][j] += -2*DbarDbarphi[i][j] + 4*der(symb_phi,i)*der(symb_phi,j)
        RphiLL[j][i] += -2*DbarDbarphi[i][j] + 4*der(symb_phi,i)*der(symb_phi,j)
        for l in range(len(directions)):
            for m in range(l,len(directions)):
                RphiLL[i][j] += 2*symb_gbarLL[i][j]*symb_gbarUU[l][m]*(-2*DbarDbarphi[l][m]-4*der(symb_phi,l)*der(symb_phi,m))
                RphiLL[j][i] += 2*symb_gbarLL[i][j]*symb_gbarUU[l][m]*(-2*DbarDbarphi[l][m]-4*der(symb_phi,l)*der(symb_phi,m))

In [16]:
RLL = ixp.zerorank2(3)
for i in range(len(directions)):
    for j in range(i,len(directions)):
        RLL[i][j] += RbarLL[i][j]+RphiLL[i][j]
        RLL[j][i] += RbarLL[i][j]+RphiLL[i][j]

As the case for the inverse metric, we will define intermediate variables for the Ricci tensor and its trace free part, to reduce the number of variable calls in the code.

In [17]:
symb_RLL = ixp.declarerank2('RLL','sym01')
#for itri in range(len(name_gUU)):
 #   for itrj in range(itri,len(name_gUU)):
  #      var_names.append(str(name_gUU[itri][itrj]))
symb_RLL = np.array(symb_RLL)

In [18]:
symb_RTFLL = ixp.declarerank2('RTFLL','sym01')
#for itri in range(len(name_gUU)):
 #   for itrj in range(itri,len(name_gUU)):
  #      var_names.append(str(name_gUU[itri][itrj]))
symb_RTFLL = np.array(symb_RTFLL)

In [19]:
Rsclr = 0
for i in range(3):
    for j in range(3):
        Rsclr += symb_gbarUU[i][j]*symb_RLL[i][j]

In [20]:
symb_Rsclr = symbols('Rsclr')

Right hand side of the equations of motion for phi,and metric.

In [21]:
RHS_phi = -1/6*symb_alpha*symb_Ksclr
for k in range(len(directions)):
    RHS_phi += symb_betaU[k]*der(symb_phi,k)+1/6*der(symb_betaU[k],k)

In [22]:
RHS_gbarLL = ixp.zerorank2(3)
for i in range(3):
    for j in range(3):
        RHS_gbarLL[i][j] += -2*symb_alpha*symb_AbarLL[i][j]
        for k in range(3):
            RHS_gbarLL[i][j] += symb_betaU[k]*der(symb_gbarLL[i][j],k)+symb_gbarLL[i][k]*der(symb_betaU[k],j)+symb_gbarLL[j][k]*der(symb_betaU[k],i)-2/3*symb_gbarLL[i][j]*der(symb_betaU[k],k)

Intermediate variable definitions for Abar with raised indices.  Will make defining EOMS for Ksclr and Abar below.

In [23]:
symb_AbarUU = ixp.zerorank2(3)
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                symb_AbarUU[i][j] += symb_gbarUU[i][k]*symb_gbarUU[j][l]*symb_AbarLL[k][l]
                
symb_AbarUL = ixp.zerorank2(3)
for i in range(3):
    for j in range(3):
        for k in range(3):
            symb_AbarUL[i][j] = symb_gbarUU[i][k]*symb_AbarLL[k][j]

Second covariant derivatives of alpha defined.

In [24]:
DDalpha= ixp.zerorank2(3)
for i in range(3):
    for j in range(i,3):
        DDalpha[i][j] += der(der(symb_alpha,i),j)
        DDalpha[j][i] += der(der(symb_alpha,i),j)
        for k in range(3):
            DDalpha[i][j] += -GammaULL[k][i][j]*der(symb_alpha,k)
            DDalpha[j][i] += -GammaULL[k][i][j]*der(symb_alpha,k)

In [25]:
RHS_Ksclr=0
for i in range(3):
    for j in range(3):
        RHS_Ksclr += -sp.exp(4*symb_phi)*symb_gbarLL[i][j]*DDalpha[i][j]
        RHS_Ksclr += symb_alpha*(symb_AbarLL[i][j]*symb_AbarUU[i][j]+1/3*symb_Ksclr**2)
for i in range(3):
    RHS_Ksclr += symb_betaU[i]*der(symb_Ksclr,i)

In [26]:
DDalphaTF = ixp.zerorank2(3)
for i in range(3):
    for j in range(3):
        DDalphaTF[i][j]=DDalpha[i][j]
        for l in range(3):
            for m in range(3):
                DDalphaTF[i][j] += -1/3*sp.exp(4*symb_phi)*symb_gbarLL[i][j]*symb_gbarUU[l][m]*DDalpha[l][m]
        

Intermediate definition of the trace free Ricci tensor. 

In [27]:
RTFLL = ixp.zerorank2(3)
for i in range(3):
    for j in range(i,3):
        RTFLL[i][j] = symb_RLL[i][j]
        RTFLL[j][i] = symb_RLL[i][j]
        for l in range(3):
            for m in range(l,3):
                RTFLL[i][j] += -2/3*sp.exp(4*symb_phi)*symb_gbarLL[i][j]*symb_gbarUU[l][m]*symb_RLL[l][m]
                RTFLL[j][i] += -2/3*sp.exp(4*symb_phi)*symb_gbarLL[i][j]*symb_gbarUU[l][m]*symb_RLL[l][m]

RHS of AbarLL finally defined.

In [28]:
RHS_AbarLL = ixp.zerorank2(3)
for i in range(3):
    for j in range(i,3):
        RHS_AbarLL[i][j] += sp.exp(-4*symb_phi)*(-DDalphaTF[i][j]+symb_alpha*(symb_RTFLL[i][j]))+symb_alpha*symb_Ksclr*symb_AbarLL[i][j]
        RHS_AbarLL[j][i] += sp.exp(-4*symb_phi)*(-DDalphaTF[i][j]+symb_alpha*(symb_RTFLL[i][j]))+symb_alpha*symb_Ksclr*symb_AbarLL[i][j]
        for k in range(3):
            RHS_AbarLL[i][j] += -2*symb_alpha*symb_AbarLL[i][k]*symb_AbarUL[k][j]
            RHS_AbarLL[i][j] += symb_betaU[k]*der(symb_AbarLL[i][j],k)+symb_AbarLL[i][k]*der(symb_betaU[k],j)+symb_AbarLL[k][j]*der(symb_betaU[k],i)-2/3*symb_AbarLL[i][j]*der(symb_betaU[k],k)
            RHS_AbarLL[j][i] += -2*symb_alpha*symb_AbarLL[i][k]*symb_AbarUL[k][j]
            RHS_AbarLL[j][i] += symb_betaU[k]*der(symb_AbarLL[i][j],k)+symb_AbarLL[i][k]*der(symb_betaU[k],j)+symb_AbarLL[k][j]*der(symb_betaU[k],i)-2/3*symb_AbarLL[i][j]*der(symb_betaU[k],k)


RHS of the conformal connection coefficients.

In [29]:
RHS_GambarU = ixp.zerorank1(3)
for i in range(3):
    for j in range(3):
        RHS_GambarU[i] += -2*symb_AbarUU[i][j]*der(symb_alpha,j)+2*symb_alpha*(-2/3*symb_gbarUU[i][j]*der(symb_Ksclr,j)+6*symb_AbarUU[i][j]*der(symb_phi,j))
        RHS_GambarU[i] += symb_betaU[j]*der(symb_GambarU[i],j)-symb_GambarU[j]*der(symb_betaU[i],j)+2/3*symb_GambarU[i]*der(symb_betaU[j],j)
        for l in range(j,3):
            RHS_GambarU[i] += 4*symb_alpha*GammabarULL[i][j][l]*symb_AbarUU[l][j]
            RHS_GambarU[i] +=2/3*symb_gbarUU[l][i]*der(der(symb_betaU[j],j),l)+symb_gbarUU[l][j]*der(der(symb_betaU[i],j),l)

RHS for the gauge variables alpha, beta, and B in the 1+log and Gamma Driver schemes.

In [30]:
RHS_alpha = -2*symb_alpha*symb_Ksclr
for i in range(3):
    RHS_alpha += symb_betaU[i]*der(symb_alpha,i)

In [31]:
RHS_betaU = ixp.zerorank1(3)
for i in range(3):
    RHS_betaU[i] += 3/4*symb_BU[i]
    for j in range(3):
        RHS_betaU[i] += symb_betaU[j]*der(symb_betaU[i],j)

In [32]:
RHS_BU = ixp.zerorank1(3)
eta = 1
for i in range(3):
    RHS_BU[i] += RHS_GambarU[i]-eta*symb_BU[i]
    for j in range(3):
        RHS_BU[i] += symb_betaU[j]*der(symb_BU[i],j)

We define the constraint equations.

In [44]:
ConstraintH = 0
for i in range(3):
    for j in range(i,3):
        ConstraintH += 2*sp.exp(symb_phi)*symb_gbarUU[i][j]*(der(der(symb_phi,i),j)+DbarDbarphi[i][j])+sp.exp(5*symb_phi)*symb_AbarLL[i][j]*symb_AbarUU[i][j]/4
ConstraintH +=sp.exp(symb_phi)*symb_Rsclr/8-sp.exp(symb_phi)*(symb_Ksclr**2)/12

ConstraintH = 0
for i in range(3):
        ConstraintH += sp.exp(symb_phi)*(der(symb_phi,i)*der(symb_phi,i)+der(der(symb_phi,i),i))

In [45]:
AMReXcode(ConstraintH)

'(ddphi00 + std::pow(dphi0, 2))*std::exp(phi) + (ddphi11 + std::pow(dphi1, 2))*std::exp(phi) + (ddphi22 + std::pow(dphi2, 2))*std::exp(phi);'

In [35]:
ConstraintM = ixp.zerorank1(3)
for i in range(3):
    for j in range(3):
        ConstraintM[i] += -2/3*sp.exp(6*symb_phi)*symb_gbarUU[i][j]*der(symb_Ksclr,j)
for i in range(3):
    for j in range(3):
        ConstraintM[i] += sp.exp(6*symb_phi)*6*der(symb_phi,j)*symb_AbarUU[j][i]

DbarAbarLLL = ixp.zerorank3(3)
for j in range(3):
    for k in range(3):
        for l in range(3):
            DbarAbarLLL[j][k][l] += der(symb_AbarLL[k][l],j)
            for q in range(3):
                DbarAbarLLL[j][k][l] += -GammabarLLL[q][j][k]*symb_AbarLL[q][l]-GammabarLLL[q][j][l]*symb_AbarLL[k][q]
            
        
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                ConstraintM[i] +=sp.exp(6*symb_phi)*symb_gbarUU[j][k]*symb_gbarUU[i][l]*DbarAbarLLL[j][k][l]

We must initialize all of the the evolved variables in the list. 

In [36]:
x,y,z = sp.symbols('x y z')
r = sp.sqrt(x**2 + y**2 + z**2)

In [37]:
list_init = []

M = 1

phi_init = sp.log(1+M/(2*r))
list_init.append(phi_init)

gbarLL_init = np.array(sp.eye(3))
for i in range(3):
    for j in range(i,3):
        list_init.append(gbarLL_init[i][j])
        
Ksclr_init = 0
list_init.append(Ksclr_init)

AbarLL_init = np.array([[0,0,0],[0,0,0],[0,0,0]])
gbarLL_init = np.array(sp.eye(3))
for i in range(3):
    for j in range(i,3):
        list_init.append(AbarLL_init[i][j])

GambarU_init = np.array([0,0,0])
for i in range(3):
    list_init.append(GambarU_init[i])
    
alpha_init = (1-M/(2*r))/(1+M/(2*r))
list_init.append(alpha_init)

betaU_init = np.array([0,0,0])
for i in range(3):
    list_init.append(betaU_init[i])
    
BU_init = np.array([0,0,0])
for i in range(3):
    list_init.append(BU_init[i])

We now create the AMReX files for the equations of motion.  I do not recommend looking at the RHS file generated.

In [38]:
Difforder = 4

In [39]:
path = "../Source/BSSN/"

In [47]:
list_diags = ['H']

In [40]:
createSETUP(path+"ET_Integration_Setup.H",list_vars,list_diags,2)

In [41]:
createVARIABLES(path+"ET_Integration_Variables.H",list_vars)

In [42]:
createINIT(path+"ET_Integration_Init_K.H")
for itr in range(len(list_vars)):
    addINIT(path+"ET_Integration_Init_K.H",AMReXcode(list_init[itr],list_vars,declare_state=True, statename=list_vars[itr]))
finishINIT(path+"ET_Integration_Init_K.H")

The evolved variables, intermediate variables, and derivatives are all initialized in the code as intermediated AMReX reals defined in terms of the grid functions (multifabs).  The RHS equations we've coded above are all written in terms of these to reduce the number of calls to the grid data in the EOMS.

In [43]:
createRHS(path+"ET_Integration_Rhs_K.H")
fileRHS=open(path+"ET_Integration_Rhs_K.H","a+")
fileRHS.write("         amrex::Real " + str(symb_phi) + " = " + AMReXcode(phi,list_vars)+'\n')

for i in range(3):
    for j in range(i,3):
        fileRHS.write("         amrex::Real " + str(symb_gbarLL[i][j]) + " = " + AMReXcode(gbarLL[i][j],list_vars)+'\n')

fileRHS.write("         amrex::Real " + str(symb_Ksclr) + " = " + AMReXcode(Ksclr,list_vars)+'\n')

for i in range(3):
    for j in range(i,3):
        fileRHS.write("         amrex::Real " + str(symb_AbarLL[i][j]) + " = " + AMReXcode(AbarLL[i][j],list_vars)+'\n')

for i in range(3):
    fileRHS.write("         amrex::Real " + str(symb_GambarU[i]) + " = " + AMReXcode(GambarU[i],list_vars)+'\n')

fileRHS.write("         amrex::Real " + str(symb_alpha) + " = " + AMReXcode(alpha,list_vars)+'\n')
    
for i in range(3):
    fileRHS.write("         amrex::Real " + str(symb_betaU[i]) + " = " + AMReXcode(betaU[i],list_vars)+'\n')
    
for i in range(3):
    fileRHS.write("         amrex::Real " + str(symb_BU[i]) + " = " + AMReXcode(BU[i],list_vars)+'\n')
    
for i in range(3):
    for j in range(i,3):
        fileRHS.write("         amrex::Real " + str(symb_gbarUU[i][j]) + " = " + AMReXcode(sp.simplify(sp.Matrix(symb_gbarLL).inv()[i,j]))+'\n')

for i in range(3):
    fileRHS.write("         amrex::Real " + str(der(symb_phi,i)) + " = " + AMReXcode(Diff1(phi,'dD'+str(i),Difforder),list_vars)+'\n')
    
for i in range(3):
    for j in range(i,3):
        fileRHS.write("         amrex::Real " + str(der(der(symb_phi,i),j)) + " = " + AMReXcode(Diff2(phi,'dDD'+str(i)+str(j),Difforder),list_vars)+'\n')

for i in range(3):
    fileRHS.write("         amrex::Real " + str(der(symb_Ksclr,i)) + " = " + AMReXcode(Diff1(Ksclr,'dD'+str(i),Difforder),list_vars)+'\n')
        
for i in range(3):
    fileRHS.write("         amrex::Real " + str(der(symb_alpha,i)) + " = " + AMReXcode(Diff1(alpha,'dD'+str(i),Difforder),list_vars)+'\n')    

for i in range(3):
    for j in range(i,3):
        fileRHS.write("         amrex::Real " + str(der(der(symb_alpha,i),j)) + " = " + AMReXcode(Diff2(alpha,'dDD'+str(i)+str(j),Difforder),list_vars)+'\n')
    
for i in range(3):
    for j in range(3):
        fileRHS.write("         amrex::Real " + str(der(symb_betaU[i],j)) + " = " + AMReXcode(Diff1(betaU[i],'dD'+str(j),Difforder),list_vars)+'\n')
        
for i in range(3):
    for j in range(3):
        for k in range(j,3):
            fileRHS.write("         amrex::Real " + str(der(der(symb_betaU[i],j),k)) + " = " + AMReXcode(Diff2(betaU[i],'dDD'+str(j)+str(k),Difforder),list_vars)+'\n')

for i in range(3):
    for j in range(3):
        fileRHS.write("         amrex::Real " + str(der(symb_BU[i],j)) + " = " + AMReXcode(Diff1(BU[i],'dD'+str(j),Difforder),list_vars)+'\n')
            
for i in range(3):
    for j in range(i,3):
        for k in range(3):
            fileRHS.write("         amrex::Real " + str(der(symb_gbarLL[i][j],k)) + " = " + AMReXcode(Diff1(gbarLL[i][j],'dD'+str(k),Difforder),list_vars)+'\n')

for i in range(3):
    for j in range(i,3):
        for k in range(3):
            for l in range(k,3):
                fileRHS.write("         amrex::Real " + str(der(der(symb_gbarLL[i][j],k),l)) + " = " + AMReXcode(Diff2(gbarLL[i][j],'dDD'+str(k)+str(l),Difforder),list_vars)+'\n')

for i in range(3):
    for j in range(i,3):
        for k in range(3):
            fileRHS.write("         amrex::Real " + str(der(symb_AbarLL[i][j],k)) + " = " + AMReXcode(Diff1(AbarLL[i][j],'dD'+str(k),Difforder),list_vars)+'\n')
                
for i in range(3):
    for j in range(3):
        fileRHS.write("         amrex::Real " + str(der(symb_GambarU[i],j)) + " = " + AMReXcode(Diff1(GambarU[i],'dD'+str(j),Difforder),list_vars)+'\n')
        
for i in range(3):
    for j in range(i,3):
        fileRHS.write("         amrex::Real " + str(symb_RLL[i][j]) + " = " + AMReXcode(RLL[i][j])+'\n')
        
for i in range(3):
    for j in range(i,3):
        fileRHS.write("         amrex::Real " + str(symb_RTFLL[i][j]) + " = " + AMReXcode(RTFLL[i][j])+'\n')
    
fileRHS.close()

addRHS(path+"ET_Integration_Rhs_K.H",AMReXcode(RHS_phi,declare_rhs = True, rhsname = str(symb_phi)))

for i in range(3):
    for j in range(i,3):
        addRHS(path+"ET_Integration_Rhs_K.H",AMReXcode(RHS_gbarLL[i][j],declare_rhs = True, rhsname = str(symb_gbarLL[i][j])))

addRHS(path+"ET_Integration_Rhs_K.H",AMReXcode(RHS_Ksclr,declare_rhs = True, rhsname = str(symb_Ksclr)))

for i in range(3):
    for j in range(i,3):
        addRHS(path+"ET_Integration_Rhs_K.H",AMReXcode(RHS_AbarLL[i][j],declare_rhs = True, rhsname = str(symb_AbarLL[i][j])))

for i in range(3):
    addRHS(path+"ET_Integration_Rhs_K.H",AMReXcode(RHS_GambarU[i],declare_rhs = True, rhsname = str(symb_GambarU[i])))

addRHS(path+"ET_Integration_Rhs_K.H",AMReXcode(RHS_alpha,declare_rhs = True, rhsname = str(symb_alpha)))

for i in range(3):
    addRHS(path+"ET_Integration_Rhs_K.H",AMReXcode(RHS_betaU[i],declare_rhs = True, rhsname = str(symb_betaU[i])))
    
for i in range(3):
    addRHS(path+"ET_Integration_Rhs_K.H",AMReXcode(RHS_BU[i],declare_rhs = True, rhsname = str(symb_BU[i])))

finishRHS(path+"ET_Integration_Rhs_K.H")